# Mansa loans

## EDA

We start by performing an EDA of the datasets.

In [1]:
import pandas as pd
from src.data_tools import get_data

In [2]:
# read the csv files
df_accounts = get_data("data/accounts.csv")
df_transactions = get_data("data/transactions.csv")

In [3]:
df_accounts.head()

,id,balance,update_date
0,0,13.63,2021-07-02
1,1,12.91,2021-07-02
2,2,19.84,2021-07-02
3,3,130.00,2021-07-02
4,4,2806.75,2021-07-02


In [4]:
df_transactions.head()

,account_id,date,amount
0,0,2020-10-16,200.0
1,0,2020-10-16,-192.0
2,0,2020-10-16,200.0
3,0,2020-10-16,-24.0
4,0,2020-10-16,-50.0


#### Checking for null values

In [5]:
df_accounts.isna().any()

id             False
balance        False
update_date    False
dtype: bool

In [6]:
df_transactions.isna().any()

account_id    False
date          False
amount        False
dtype: bool

There is no null value in any of the two datasets.


#### Checking for duplicates

In [7]:
df_accounts.duplicated().any()

False

In [8]:
df_transactions.duplicated().any()

True

There are duplicates in the transactions. Lets look at some of them

In [9]:
n_duplicated = df_transactions.duplicated().sum()
dup_percent = int(n_duplicated / df_transactions.shape[0]*100)
print(f'there are {n_duplicated} duplicates in transactions.csv, representing {dup_percent}% of the data.')

there are 28661 duplicates in transactions.csv, representing 5% of the data.


In [10]:
df_dup = df_transactions[df_transactions.duplicated()]
df_dup.head()

,account_id,date,amount
2,0,2020-10-16,200.0
11,0,2020-10-17,0.0
14,0,2020-10-19,0.0
23,0,2020-10-21,0.0
45,0,2020-10-29,-5.5


In [11]:
df_transactions[
    (df_transactions.account_id == 0) 
    & (df_transactions.date == '2020-10-16') 
    &(df_transactions.amount == 200.0)]

,account_id,date,amount
0,0,2020-10-16,200.0
2,0,2020-10-16,200.0


It is difficult, without further information if the above transactions are legitimate siilar transactions or duplicates coming from an error when the data were processed. We decide to keep these duplicates. It would be interesting to enrich the transactions data with a transaction id. We could be certain if a similar transaction on a given account and a given date is a duplicate or legitimate.

#### Account history distribution

Let's check the history distribution of the different accounts in our data.

In [12]:
# we change the date column from str to datetime
df_transactions['date'] = pd.to_datetime(df_transactions['date'])
group = df_transactions.groupby('account_id')['date']

In [13]:
history_df = group.agg(["min", "max"])
history = history_df['max'] - history_df['min']
history.head()

account_id
0   257 days
1   231 days
2   100 days
3   176 days
4   293 days
dtype: timedelta64[ns]

In [14]:
trigger = pd.Timedelta(180, "d")
n_accounts = (history > trigger).sum()
frac_long_account = int(n_accounts / len(history) * 100)
print(f'{n_accounts} accounts, which represent {frac_long_account}% of all the accounts, have an history of more than 6 months')

823 accounts, which represent 65% of all the accounts, have an history of more than 6 months


We'll focus on the accounts with more than 6 months history. For that purpose we use get_df_with_history.

In [15]:
from src.data_tools import get_df_with_history
df_accounts, df_transactions = get_df_with_history(df_accounts, df_transactions)

In [16]:
n_transac = len(set(df_transactions['account_id']))
print(f'We are left with {len(df_accounts)} accounts in df_accounts and {n_transac} in df_transactions which is as expected.')

We are left with 860 accounts in df_accounts and 860 in df_transactions which is as expected.


## Processing the data for training

We are looking to predict the next month outgoing given the last 6 month of transaction. For that purpose it make sense to divide the history of transactions in 30 days buckets and to calculate the total amount of inflow and outflow for each 30 days bucket.  
When processing the data, we keep the last 2 months for testing and the previous 2 months for validation so that we are sure there is no data leakage when training and testing. 

In [17]:
from src.data_tools import get_training_data
# When processing the data, we keep the last 2 months for testing and the previous 2 months for validation
training_data = get_training_data(test_size=2)

In [18]:
training_data.keys()

dict_keys(['train', 'val', 'test'])

In [19]:
training_data['train'].keys()

dict_keys(['X', 'y'])

The data are split between training, validation and test. Each of these split is a dictionary with the input of the model: 'X' and the target variable: 'y'. Let's check the relative proportion of these different splits:

In [20]:
n_train = training_data['train']['X'].shape[0]
n_val = training_data['val']['X'].shape[0]
n_test = training_data['test']['X'].shape[0]
n = n_train + n_val + n_test
print(f'There is a total of {n} data')
print(f'{n_train/n*100:.1f}% for training, {n_val/n*100:.1f}% for validation and {n_test/n*100:.1f}% for testing')

There is a total of 13520 data
75.5% for training, 12.3% for validation and 12.2% for testing


This is an acceptable split of the data for training our model. Lets look at how the data were processed:

In [21]:
training_data['train']['X'].head()

,1M inflow,2M inflow,3M inflow,4M inflow,5M inflow,6M inflow,1M outflow,2M outflow,3M outflow,4M outflow,5M outflow,6M outflow,initial_balance
0,90.00,0.00,0.00,50.00,0.00,0.00,0.00,-50.00,-50.00,-100.00,-50.00,-50.00,2525.00
1,4138.47,5376.46,3833.00,3600.00,3097.20,2000.00,-4946.87,-4929.89,-3876.41,-4195.21,-2071.26,-1680.31,70.20
2,0.00,0.00,0.00,0.00,1182.00,4034.00,0.00,0.00,0.00,0.00,-21.27,-2088.17,-80.94
3,1705.00,646.00,231.85,312.35,455.07,4787.98,-1351.20,-1039.65,-217.80,-255.07,-271.26,-754.99,280.66
4,0.00,0.00,0.00,0.00,1040.00,714.00,0.00,0.00,0.00,0.00,0.00,-68.65,123.69


These data represent 6 months of transaction history. For example, '1M inflow' gives the total amount of positive transactions in the accounts during the first month of the history considered and '6M outflow' gives the total amount of negative transaction in the accounts during the last month of the history considered. initial_balance give the balances of the account at the beginning of the 6 months period considered.

We have organised the data as needed for the next step:

In [22]:
X_train = training_data['train']['X']
y_train = training_data['train']['y']
X_val = training_data['val']['X']
y_val = training_data['val']['y']
X_test = training_data['test']['X']
y_test = training_data['test']['y']


## Model choice

Given that time is limited we'll compare different model performance with their default setting to choose our model

In [23]:
# we perform some scaling on the data. It is not necessary for tree based models but we'll keep a common 
# groundwork for our analysis
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

### Regression

Regression will give us a benchmark. It's unlikely to be the most performant model, we should be able to do better with ensemble models.

In [24]:
from sklearn.linear_model import LinearRegression

# we start with linear regression
reg = LinearRegression()
reg.fit(X_train, y_train)
r2 = reg.score(X_val, y_val)
print(f'Linear regression regression r2: {r2}')

Linear regression regression r2: 0.26590342126231004


In [25]:
from sklearn.linear_model import Ridge

# we impose a penalty using Ridge regression
reg = Ridge()
reg.fit(X_train, y_train)
r2 = reg.score(X_val, y_val)
print(f'Ridge regression r2: {r2}')

Ridge regression r2: 0.26592745106054116


In [26]:
from sklearn.linear_model import Lasso

# we now impose a penalty using Lasso regression
reg = Lasso()
reg.fit(X_train, y_train)
r2 = reg.score(X_val, y_val)
print(f'Lasso regression r2: {r2}')

Lasso regression r2: 0.2658387256598487


### Random Forests

In [27]:
from sklearn.ensemble import RandomForestRegressor
reg_rf = RandomForestRegressor(n_estimators=100, random_state=48)
reg_rf.fit(X_train, y_train)
r2 = reg_rf.score(X_val, y_val)
print(f'Random forest r2: {r2}')

Random forest r2: 0.1799939470557319


### Light GBM

In [28]:
from lightgbm import LGBMRegressor
reg_lg = LGBMRegressor(n_estimators=100, random_state=48)
reg_lg.fit(X_train, y_train)
r2 = reg_lg.score(X_val, y_val)
print(f'Light gbm r2: {r2}')

Light gbm r2: 0.28423212148743804


Light GBM seems the most promising model. We'll use Light GBM from now on

### Hyperparameter tuning for Light GBM

We use Optuna for hyperparameters tuning and fine tune more parameters:
- parameters that control the tree structure: num_leaves and max_depth (between 3 and 12). LGBM documentation indicates that num_leaves should be < 2^(max_depth)
- parameters for better accuracy: n_estimators and learning_rate (0.01-0.3)
- parameters to control overfitting: regularization (l1 or l2) and bagging fraction  


In [29]:
import optuna
from src.hyper_params import get_objective

/Users/gdegournay/opt/anaconda3/envs/mansa/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
study = optuna.create_study(direction='minimize')
objective = get_objective(X_train, X_val, y_train, y_val)
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-04-22 14:04:30,452] A new study created in memory with name: no-name-7c9d3e30-7753-4e05-9d04-f2768c9a000b


Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 7660.11
[200]	valid_0's rmse: 7386.87


[I 2022-04-22 14:04:30,804] Trial 0 finished with value: 7381.3500106699985 and parameters: {'n_estimators': 1250, 'reg_alpha': 0.718567010048906, 'reg_lambda': 0.30879783323044596, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.01, 'max_depth': 12, 'num_leaves': 423, 'min_child_samples': 139, 'min_data_per_groups': 36}. Best is trial 0 with value: 7381.3500106699985.


[300]	valid_0's rmse: 7420.32
Early stopping, best iteration is:
[228]	valid_0's rmse: 7381.35
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 7959.14


[I 2022-04-22 14:04:31,131] Trial 1 finished with value: 7521.91418835897 and parameters: {'n_estimators': 1000, 'reg_alpha': 0.5010106557156396, 'reg_lambda': 0.020814518035434534, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.008, 'max_depth': 12, 'num_leaves': 233, 'min_child_samples': 201, 'min_data_per_groups': 83}. Best is trial 0 with value: 7381.3500106699985.


...

[200]	valid_0's rmse: 7300.72
Early stopping, best iteration is:
[175]	valid_0's rmse: 7295.01
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 8009.9


[I 2022-04-22 14:04:43,398] Trial 48 finished with value: 7313.983923411407 and parameters: {'n_estimators': 500, 'reg_alpha': 0.0010005607086849629, 'reg_lambda': 0.06283525699766657, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.006, 'max_depth': 7, 'num_leaves': 532, 'min_child_samples': 57, 'min_data_per_groups': 66}. Best is trial 8 with value: 7263.847092796182.


[200]	valid_0's rmse: 7410.99
[300]	valid_0's rmse: 7316.4
Early stopping, best iteration is:
[282]	valid_0's rmse: 7313.98
Training until validation scores don't improve for 100 rounds


[I 2022-04-22 14:04:43,666] Trial 49 finished with value: 7370.233440136633 and parameters: {'n_estimators': 750, 'reg_alpha': 0.003589946351202035, 'reg_lambda': 0.024026758117131126, 'colsample_bytree': 0.9, 'subsample': 0.5, 'learning_rate': 0.008, 'max_depth': 7, 'num_leaves': 736, 'min_child_samples': 81, 'min_data_per_groups': 34}. Best is trial 8 with value: 7263.847092796182.


[100]	valid_0's rmse: 7736.7
[200]	valid_0's rmse: 7386.77
[300]	valid_0's rmse: 7400.8
Early stopping, best iteration is:
[240]	valid_0's rmse: 7370.23
Number of finished trials: 50
Best trial: {'n_estimators': 1000, 'reg_alpha': 0.0016119218181259286, 'reg_lambda': 0.002509094868592898, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 0.006, 'max_depth': 7, 'num_leaves': 436, 'min_child_samples': 23, 'min_data_per_groups': 62}


Let's create an LGBMRegresor with the best hyperparameters found in the previous study

In [31]:
params=study.best_params   
params['random_state'] = 48
params['metric'] = 'rmse'
params

{'n_estimators': 1000,
 'reg_alpha': 0.0016119218181259286,
 'reg_lambda': 0.002509094868592898,
 'colsample_bytree': 0.9,
 'subsample': 0.7,
 'learning_rate': 0.006,
 'max_depth': 7,
 'num_leaves': 436,
 'min_child_samples': 23,
 'min_data_per_groups': 62,
 'random_state': 48,
 'metric': 'rmse'}

In [32]:
_ = params.pop('min_data_per_groups') # no such parameter in light gbm
model = LGBMRegressor(**params)
model.fit(X_train,y_train)
r2 = model.score(X_val, y_val)


In [33]:
r2 = model.score(X_val, y_val)
print(f'Light gbm r2 after hyper parameters tuning: {r2}')

Light gbm r2 after hyper parameters tuning: 0.39505408879971415


In [34]:
from src.model import save_model, load_model

In [38]:
save_model(model,'lgb2')

In [39]:
m = load_model('lgb2')

In [40]:
r2 = m.score(X_val, y_val)

In [41]:
r2

0.39505408879971415